Code is adapted from [Huggingface's Task Guide](https://huggingface.co/docs/peft/task_guides/clm-prompt-tuning)

In [ ]:
%%capture
!pip install -q peft transformers datasets

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, PeftModel, PeftConfig
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
import re
import math

In [ ]:
%%capture
dataset_name="sst2"
dataset = load_dataset(dataset_name)

In [ ]:
original_eval_labels = dataset["validation"][label_column]

In [ ]:
device = "cuda"
model_name_or_path = "bigscience/bloomz-1b1"
tokenizer_name_or_path = "bigscience/bloomz-1b1"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify the sentiment of the film review as positive or negative:",
    tokenizer_name_or_path=model_name_or_path,
)

checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)

text_column = "sentence"
label_column = "label"
classes = ["negative", "positive"]
max_length = 64
lr = 1e-4
num_epochs = 2
batch_size = 8

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left")
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)

1


In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [classes[x] for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/67349 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]
testing_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(testing_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

trainable params: 12,288 || all params: 1,065,326,592 || trainable%: 0.001153449101174788
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/8419 [00:00<?, ?it/s]

  0%|          | 0/228 [00:00<?, ?it/s]

epoch=0: train_ppl=tensor(139.2792, device='cuda:0') train_epoch_loss=tensor(4.9365, device='cuda:0') eval_ppl=tensor(238.0433, device='cuda:0') eval_epoch_loss=tensor(5.4725, device='cuda:0')


  0%|          | 0/8419 [00:00<?, ?it/s]

  0%|          | 0/228 [00:00<?, ?it/s]

epoch=1: train_ppl=tensor(109.8711, device='cuda:0') train_epoch_loss=tensor(4.6993, device='cuda:0') eval_ppl=tensor(201.4124, device='cuda:0') eval_epoch_loss=tensor(5.3054, device='cuda:0')


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
peft_model_id = "fabiopernisi/bloomz-1b1_PROMPT_TUNING_CAUSAL_LM_sst2_2"
model.push_to_hub("fabiopernisi/bloomz-1b1_PROMPT_TUNING_CAUSAL_LM_sst2_2", use_auth_token=True, create_pr=1)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:831: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/49.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fabiopernisi/bloomz-1b1_PROMPT_TUNING_CAUSAL_LM_sst2_2/commit/dd3f18c17f3e2ff131701807e52eff6c37fe75ec', commit_message='Upload model', commit_description='', oid='dd3f18c17f3e2ff131701807e52eff6c37fe75ec', pr_url='https://huggingface.co/fabiopernisi/bloomz-1b1_PROMPT_TUNING_CAUSAL_LM_sst2_2/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [ ]:
peft_model_id = "fabiopernisi/bloomz-1b1_PROMPT_TUNING_CAUSAL_LM_sst2_2"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Testing on a single instance of the test-set

In [ ]:
inputs = tokenizer(
    f"{text_column} : '{{I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered \"controversial\" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it\\'s not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn\\'t have much of a plot.}}' Label : ",
    return_tensors="pt",
)

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

Testing on the whole dataset

In [ ]:
test_dataloader = DataLoader(testing_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [ ]:
model.to(device)

all_predictions = []
all_true_labels = []

for i, batch in enumerate(tqdm(test_dataloader)):
    input_ids = batch['input_ids']
    inputs_texts = [tokenizer.decode(input_id, skip_special_tokens=True).split(" Label : ")[0] for input_id in input_ids]
    texts = [input_text.split(f"{text_column} : ")[1] for input_text in inputs_texts]

    true_labels_indices = original_eval_labels[i * batch_size: (i + 1) * batch_size]
    true_labels = [classes[label] for label in true_labels_indices]

    for text, true_label in zip(texts, true_labels):
        inputs = tokenizer(
            f"{text_column} : {text} Label : ",
            return_tensors="pt",
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        input_length = inputs["input_ids"].shape[-1]
        max_output_length = 20
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=input_length + max_output_length,
                num_return_sequences=1,
                repetition_penalty=1.2,
                temperature=0.4,
                do_sample=True,
            )
            predicted_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

            predicted_label_part = predicted_text.split("Label : ")[-1].strip()

            predicted_label = None
            for class_name in classes:
                if class_name in predicted_label_part:
                    predicted_label = class_name
                    break

            if predicted_label:
                all_predictions.append(predicted_label)
                all_true_labels.append(true_label)
            else:
                print(f"Unexpected label {predicted_label_part} in:", predicted_text)


  0%|          | 0/109 [00:00<?, ?it/s]

In [ ]:
accuracy = accuracy_score(all_true_labels, all_predictions)
print(f"Accuracy on the validation dataset: {accuracy * 100:.2f}%")

Accuracy on the validation dataset: 89.56%
